In [ ]:
#----------------------
# Mount Drive to Colab
#----------------------
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Text Retrieval and Search Engines - Homework 1 ##
Micaela Singer <br>
Tomer Hadad 
### a. Inverted Index ###

In [ ]:
import os
from nltk.stem import *
import re
from collections import defaultdict

In [ ]:

class InvertedIndex:
    def __init__(self, files_path):
        self.files_path = files_path

        # internal-external document id maps
        self.docno2docid = {}
        self.docid2docno = {}

        self.inverted_index = defaultdict(list)

        # tokens to ignore (dont add to the inverted index dictionary)
        self.blacklist = ['<DOC>', '<TEXT>', '<DOCNO>', '</DOC>', '</TEXT>', '</DOCNO>']

    def create_index(self):
        doc_id = 0
        token_docno = False

        # For every file in the dataset
        for filename in os.listdir(self.files_path):
            with open(os.path.join(self.files_path, filename), 'r') as file:
                content = file.read()

                # Split the file into tokens by \n and whitespace
                tokens = content.split()

                # For every token in the file
                for i, token in enumerate(tokens):

                    # If it is the start of a new document
                    if token == '<DOCNO>':
                        doc_id += 1

                        # Fetch the actual document id
                        curr_doc_no = tokens[i+1]

                        # Add the document internal and external ids to the map
                        self.docno2docid[curr_doc_no] = doc_id
                        self.docid2docno[doc_id] = curr_doc_no

                        # Mark the next token to ignore
                        token_docno = True

                        # Print progress
                        if doc_id % 10000 == 0:
                            print(f'reached {doc_id} documents')
                    
                    # Remove the mark
                    if token == '</DOCNO>':
                        token_docno = False
                    
                    # If the token is not marked to ignore, and not in the blacklist, and the document id is not yet saved
                    if not token_docno and token not in self.blacklist and (len(self.inverted_index[token]) == 0 or self.inverted_index[token][-1] < doc_id):

                        # Add the document id at the end of the token's list
                        self.inverted_index[token].append(doc_id)


In [ ]:
# Create the index from the dataset
inverted_index = InvertedIndex(r"/content/drive/MyDrive/Text Retrieval and Search Engines/Ex1")
inverted_index.create_index()

reached 10000 documents
reached 20000 documents
reached 30000 documents
reached 40000 documents
reached 50000 documents
reached 60000 documents
reached 70000 documents
reached 80000 documents
reached 90000 documents
reached 100000 documents
reached 110000 documents
reached 120000 documents
reached 130000 documents
reached 140000 documents
reached 150000 documents
reached 160000 documents
reached 170000 documents
reached 180000 documents
reached 190000 documents
reached 200000 documents
reached 210000 documents
reached 220000 documents
reached 230000 documents
reached 240000 documents


In [ ]:
# Classes for the query tree:

# Base class
class Node:
    def __init__(self, inverted_index, value=None, children=None, is_include=True):
        self.inverted_index = inverted_index
        self.value = value
        self.children = children
        self.is_include = is_include

    # Evaluate should be called once we would line to get the list of document ids of the query represented by the tree
    def evaluate(self):
        pass

# Word - represents a word in the query
class WORD(Node):
    def __init__(self, inverted_index, value, is_include=True):
        super().__init__(inverted_index, value, None, is_include)
    
    def evaluate(self):
        return (self.is_include, self.inverted_index[self.value])

# Or - represents an OR operator in the query
class OR(Node):
    def __init__(self, inverted_index, children):
        super().__init__(inverted_index, None, children, True)

    def evaluate(self):
        # Evaluate the children nodes to get thier lists of document ids, and wether the list should be included or excluded
        eval_children = []
        for child in self.children:
            eval_children.append(child.evaluate())

        n = len(eval_children)
        list_done = [False] * n
        pointers = [0] * n
        result = []
        done_lists = 0
        
        # While some lists still contain unexplored values
        while done_lists < n:
            min_value = float('inf')

            # Find the minimum document id between all lists and the current positions represented in 'pointers'
            for i in range(n):
                if not list_done[i] and eval_children[i][1][pointers[i]] < min_value:
                    min_value = eval_children[i][1][pointers[i]]

            # Add the document id to the result
            result.append(min_value)

            # For every document id list
            for i in range(n):

                # If we haven't reached the end of the list, and the current value is equal to the minimal value (the one added to the result)
                if not list_done[i] and eval_children[i][1][pointers[i]] == min_value:

                    # Move to the next element
                    pointers[i] += 1

                    # If we reached the end of the list, mark it as done
                    if pointers[i] >= len(eval_children[i][1]):
                        list_done[i] = True
                        done_lists += 1
            
        return (self.is_include, result)

# And - represents an AND operator in the query
class AND(Node):
    def __init__(self, inverted_index, children):
        super().__init__(inverted_index, None, children, True)

    def evaluate(self):
        # Evaluate the children nodes to get thier lists of document ids, and wether the list should be included or excluded
        eval_children = []
        for child in self.children:
            eval_children.append(child.evaluate())

        n = len(eval_children)
        pointers = [0] * n
        result = []
        list_done = [False] * n
        done_lists = 0
        
        # While some lists still contain unexplored values
        while done_lists < n:
            # Get the minimum document id of the first non-excluded list
            for i in range(n):
                if pointers[i] < len(eval_children[i][1]) and eval_children[i][0]:
                    min_value = eval_children[i][1][pointers[i]]
                    found_min_value_list_index = i
                    break

            # For every list of document ids
            for i in range(n):

                # Skip all document ids that are smaller than ours (min_value)
                while pointers[i] < len(eval_children[i][1]) and eval_children[i][1][pointers[i]] < min_value:
                    pointers[i] += 1
                
                # If we reached the end of the list, mark it as done
                if pointers[i] >= len(eval_children[i][1]):
                    list_done[i] = True
                    done_lists += 1
                
                # If the min_value fails (reached the end of an non-excluded list OR found an included list where the current value is different OR found an excluded list where the value is the same) break, and do not add the document id to the result
                if (pointers[i] == len(eval_children[i][1]) and eval_children[i][0]) or \
                   (pointers[i] < len(eval_children[i][1]) and eval_children[i][0] and eval_children[i][1][pointers[i]] != min_value) or \
                   (pointers[i] < len(eval_children[i][1]) and not eval_children[i][0] and eval_children[i][1][pointers[i]] == min_value):
                    break
            else:
                # If the for loop finished without breaking, add the document id to the result
                result.append(min_value)
            pointers[found_min_value_list_index] += 1
        
        return (self.is_include, result)

In [ ]:
class BooleanRetrieval:
    def __init__(self, inverted_index, boolean_query_file_path):
        self.inverted_index = inverted_index
        self.boolean_query_file_path = boolean_query_file_path
    
    def generate_query_tree(self, query):
        operators = ['NOT', 'AND', 'OR']
        stack = []
        tokens = query.split()
        
        # For every token in the query
        for i, token in enumerate(tokens):

            # If it is not an operator (its a word)
            if token not in operators:
                
                # Check if it should be excluded or included
                is_not = i+1 < len(tokens) and tokens[i+1] == 'NOT'

                # Add the word to the stack
                stack.append(WORD(self.inverted_index.inverted_index, token, not is_not))

            # If it is a AND operator
            if token == 'AND':

                # Empty the stack into the AND operator, and create a new stack with the new operator as the only value inside
                op = AND(self.inverted_index.inverted_index, stack)
                stack = [op]

            # If it is a OR operator
            if token == 'OR':

                # Empty the stack into the OR operator, and create a new stack with the new operator as the only value inside
                op = OR(self.inverted_index.inverted_index, stack)
                stack = [op]
        
        # If the stack has more than one node inside, put all leftover nodes into an AND operator and return
        if len(stack) > 1:
            return AND(self.inverted_index.inverted_index, stack)
        return stack[0]

    def run(self):

        # Read the queries file and create an output file
        with open(self.boolean_query_file_path, 'r') as file:
            with open('Part_2.txt', 'w') as output_file:
                lines = file.readlines()
                
                # For every query
                for i, line in enumerate(lines):
                    print(f'Line: {i+1}')

                    # Generate a query tree
                    query_tree = self.generate_query_tree(line)

                    # Evaluate the query tree
                    _, result = query_tree.evaluate()

                    # Write the external document ids into the output file
                    for docid in result:
                        output_file.write(self.inverted_index.docid2docno[docid] + ' ')
                    output_file.write('\n')

In [ ]:
boolean_retrieval = BooleanRetrieval(inverted_index, r"/content/drive/MyDrive/Text Retrieval and Search Engines/BooleanQueries.txt")
boolean_retrieval.run()

Line: 1
Line: 2
Line: 3
Line: 4
Line: 5


In [ ]:
# sort the keys by the length of their corresponding lists
sorted_keys = sorted(inverted_index.inverted_index, key=lambda k: len(inverted_index.inverted_index[k]), reverse=True)

# return the top 10 keys
top_10_keys = sorted_keys[:10]
bottom_10_keys = sorted_keys[-10:]

print(top_10_keys)
print(bottom_10_keys)

['the', 'of', 'in', 'a', 'and', 'to', 'for', 'said', 'on', 'that']
['30739', '26494', 'decertifications', '230p', '930a', 'stockcollar', 'mexpetro', '535p', 'almtriv', '118183']
